In [53]:
%run ../../common_utils.py

In [3]:
from catboost import CatBoostRegressor
import optuna
from optuna.samplers import TPESampler
from IPython.utils import io
import unidecode
from catboost import Pool

In [4]:
random_state = 1
number_of_splits = 5

In [39]:
def load_data(val_data=False,path=None,cat_unknown=False):
    train, test, metadata = load_all_data(path=path)
    nonCategorical, categorical = get_cat_and_non_cat_data(metadata)
    all_features = list(train.columns)
    all_features.remove('price')
    numerical_features = ['area_total','area_kitchen','area_living','floor','rooms','ceiling',
        'bathrooms_shared','bathrooms_private','balconies','loggias','phones','building_id','constructed','stories']
    categorical_to_numerical(train, ['street','address'])
    categorical_to_numerical(test, ['street','address'])
    if not val_data:
        X_train, y_train, test_labels = pre_process_numerical(features = all_features, numerical_features = numerical_features, train = train, test = test,
                            outliers_value=7, val_data=val_data, val_split=0.2, random_state=42, scaler="std",
                            add_R="True", add_rel_height="True", droptable=[],
                            one_hot_encode=False, cat_features=categorical, drop_old=True,categorical_unknown=cat_unknown)
        y_train_log = np.log(y_train)
        return X_train, y_train, y_train_log, test_labels
    else:
        X_train, y_train, X_test, y_test, test_labels = pre_process_numerical(features = all_features, numerical_features = numerical_features, train = train, test = test,
                            outliers_value=7, val_data=val_data, val_split=0.2, random_state=42, scaler="std",
                            add_R="True", add_rel_height="True", droptable=[],
                            one_hot_encode=False, cat_features=categorical, drop_old=True,categorical_unkown=cat_unknown)
        y_train_log = np.log(y_train)
        return X_train, y_train, y_train_log, X_test, y_test, test_labels

In [65]:
%%capture --no-display
X_train, y_train, y_train_log, test_labels = load_data(path='../',cat_unknown=True)
selected_features_catboost=['building_id','area_kitchen', 'area_living',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'phones',
       'new', 'street', 'address', 'seller', 'layout', 'condition', 'district',
       'constructed', 'stories', 'elevator_without', 'elevator_passenger',
       'material', 'parking', 'heating', 'r', 'rel_height']

X_train = X_train[selected_features_catboost]
_, _, metadata = load_all_data(path='../')
nonCategorical, categorical = get_cat_and_non_cat_data(metadata)
text_features=['street','address']
categorical.append('street')
categorical.append('address')
X_train = X_train.astype({'street':'int','address':'int','seller':'int','layout':'int','condition':'int','district':'int','material':'int','parking':'int','heating':'int'})
# X_train.drop(['area_kitchen', 'area_living', 'rooms', 'ceiling',
#        'bathrooms_shared', 'bathrooms_private', 'windows_court', 
#        'windows_street', 'balconies', 'loggias', 'phones', 'new', 'constructed',
#        'stories', 'elevator_without', 'elevator_passenger',
#        'r', 'rel_height'],axis=1,inplace=True)

In [66]:
model1 = CatBoostRegressor(random_state=random_state,loss_function='RMSE', cat_features=categorical)

average_score = catboost_groupKFold(number_of_splits, model1, X_train, y_train_log, categorical_features=[],text_features=[],random_state=random_state)

starting on split  1


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  2
starting on split  3
starting on split  4
starting on split  5


In [67]:
print("average score: ", average_score)

average score:  ([0.26136777437555175, 0.2606464150806193, 0.2462335758331197, 0.2632073672101002, 0.291418239654299], 0.26457467443073795, <catboost.core.CatBoostRegressor object at 0x00000174C73E3D60>, 4)


In [68]:
model2 = CatBoostRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    thread_count=-1,
    depth=5,
    silent=True,
    random_seed=42,
    bagging_temperature=0.2
#     od_type="Iter"
)
scores, average_score, best_model, best_index = catboost_groupKFold(number_of_splits, model2, X_train, y_train_log, categorical_features=[],text_features=categorical,random_state=random_state)

starting on split  1


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  2
starting on split  3
starting on split  4
starting on split  5


In [69]:
print("scores:        ", scores)
print("average score: ", average_score)
print("best model:    ", best_model)
print("best index:    ", best_index)

scores:         [0.28045662058144244, 0.2690561733445533, 0.24775573025631806, 0.2745180743432667, 0.32925386683584346]
average score:  0.28020809307228484
best model:     <catboost.core.CatBoostRegressor object at 0x00000174C75CD460>
best index:     4


,building_id,area_kitchen,area_living,rooms,ceiling,bathrooms_shared,bathrooms_private,windows_court,windows_street,balconies,...,district,constructed,stories,elevator_without,elevator_passenger,material,parking,heating,r,rel_height
0,-0.274350,0.057186,-0.225329,-0.170753,-0.050442,-1.090119,1.937474,0.0,1.0,-0.555415,...,11,0.791969,-0.770105,0.0,1.0,3,1,4,0.605060,-0.972520
1,-0.274350,-1.182779,-1.004520,-1.109794,-0.050442,0.310308,-1.082588,1.0,0.0,-0.555415,...,11,0.791969,-0.770105,0.0,1.0,3,1,4,0.605060,4.346979
2,-0.274350,0.821548,0.156984,0.768288,-0.050442,-1.090119,1.937474,1.0,1.0,-0.555415,...,11,0.791969,-0.770105,0.0,1.0,3,1,4,0.605060,1.509912
3,-0.274350,-1.029907,-0.942622,-1.109794,-0.050442,0.310308,-1.082588,1.0,0.0,-0.555415,...,11,0.791969,-0.770105,0.0,1.0,3,1,4,0.605060,1.509912
4,-0.274350,-1.029907,-0.942622,-1.109794,-0.050442,0.310308,-1.082588,1.0,0.0,-0.555415,...,11,0.791969,-0.770105,0.0,1.0,3,1,4,0.605060,1.509912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23280,1.354502,0.396903,-0.474379,-0.170753,-0.018514,-1.090119,0.427443,1.0,0.0,-0.555415,...,10,0.571191,-1.038344,0.0,1.0,7,3,0,2.453503,-0.617887
23281,-1.486403,-0.537318,-0.381895,-1.109794,-0.031285,0.310308,-1.082588,0.0,1.0,-0.555415,...,10,-0.532698,-0.144214,0.0,1.0,4,3,0,2.354982,-0.085937
23282,0.328451,1.042364,-0.236252,-0.170753,-0.037671,-1.090119,0.427443,1.0,0.0,-0.555415,...,7,0.350413,-0.054801,0.0,1.0,3,3,0,0.046612,1.509912
23283,-0.711220,0.074172,-0.228970,0.768288,-0.031285,0.310308,0.427443,1.0,0.0,-0.555415,...,6,-0.091142,-1.038344,0.0,1.0,7,3,4,0.497162,-0.617887
